<a href="https://colab.research.google.com/github/wolfram-laube/numopt_project-2/blob/develop/LP-TwoVars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Programming: Primal and Dual Problems



## Introduction
In linear programming, every optimization problem (primal problem) has a corresponding dual problem. Solving either problem can give insights into the other. This notebook will demonstrate the formulation and solution of both the primal and dual problems, solve them manually and computationally, and verify their equivalence.



## Problem 1

### Formulate the Primal Problem



#### Textual Formulation

**Objective Function:**
$$
\text{Minimize } Z = 3x_1 + 2x_2
$$

**Subject to:**
$$
\begin{align*}
x_1 + x_2 & \geq 4 \\
2x_1 + x_2 & \geq 5 \\
x_1 & \geq 3 \\
x_1, x_2 & \geq 0
\end{align*}
$$



#### Code Formulation

We will use the `linprog` function from the `scipy.optimize` module to solve the primal problem.


In [ ]:
import numpy as np
from scipy.optimize import linprog

# Coefficients of the objective function
c_primal = [3, 2]

# Coefficients of the inequality constraints (convert to <= form)
A_primal = [[-1, -1], [-2, -1], [-1, 0]]
b_primal = [-4, -5, -3]

# Solve the linear program
result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')

# Display the results
result_primal.x, result_primal.fun


(array([3., 1.]), 11.0)

### Formulate the Dual Problem



#### Textual Formulation

**Objective Function:**
$$
\text{Maximize } W = 4y_1 + 5y_2 + 3y_3
$$

**Subject to:**
$$
\begin{align*}
y_1 + 2y_2 + y_3 & \leq 3 \\
y_1 + y_2 & \leq 2 \\
y_1, y_2, y_3 & \geq 0
\end{align*}
$$



#### Code Formulation

We will again use the `linprog` function to solve the dual problem.


In [ ]:
# Coefficients of the objective function
c_dual = [-4, -5, -3]

# Coefficients of the inequality constraints (convert to <= form)
A_dual = [[1, 2, 1], [1, 1, 0]]
b_dual = [3, 2]

# Solve the linear program
result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')

# Display the results
result_dual.x, result_dual.fun


(array([2., 0., 1.]), -11.0)

### Manual Solution of the Primal Problem

#### Initial Simplex Tableau

To solve the primal problem manually using the Simplex method, we start by setting up the initial simplex tableau.

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & s_3 & \text{RHS} \\
\hline
s_1 & 1 & 1 & 1 & 0 & 0 & 4 \\
s_2 & 2 & 1 & 0 & 1 & 0 & 5 \\
s_3 & 1 & 0 & 0 & 0 & 1 & 3 \\
\hline
Z & -3 & -2 & 0 & 0 & 0 & 0 \\
\end{array}
$$



#### Iteration 1

- **Identify the entering variable**: The most negative coefficient in the Z-row is $-3$ for $x_1$. Thus, $x_1$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{4}{1} = 4, \quad \frac{5}{2} = 2.5, \quad \frac{3}{1} = 3
  $$
  The smallest ratio is 2.5, so $s_2$ will leave the basis.
- **Pivot**: Pivot around the element 2 in the second row, first column.



##### Pivot Steps Explained

**New Row 2 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element (2).

$$
\frac{R2}{2} \implies \frac{2}{2}, \frac{1}{2}, \frac{0}{2}, \frac{1}{2}, \frac{0}{2}, \frac{5}{2} \implies 1, \frac{1}{2}, 0, \frac{1}{2}, 0, \frac{5}{2}
$$

**New Row 1:**
Update the first row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the first row zero.

$$
R1 - R2 \implies (1, 1, 1, 0, 0, 4) - (1, \frac{1}{2}, 0, \frac{1}{2}, 0, \frac{5}{2}) \implies 0, \frac{1}{2}, 1, -\frac{1}{2}, 0, \frac{3}{2}
$$

**New Row 3:**
Update the third row similarly to make the pivot column element in the third row zero.

$$
R3 - R2 \implies (1, 0, 0, 0, 1, 3) - (1, \frac{1}{2}, 0, \frac{1}{2}, 0, \frac{5}{2}) \implies 0, -\frac{1}{2}, 0, -\frac{1}{2}, 1, \frac{1}{2}
$$

**New Z-row:**
Update the Z-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the Z-row zero.

$$
Z + 3 \times R2 \implies (-3, -2, 0, 0, 0, 0) + 3 \times (1, \frac{1}{2}, 0, \frac{1}{2}, 0, \frac{5}{2}) \implies 0, -\frac{1}{2}, 0, \frac{3}{2}, 0, \frac{15}{2}
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & s_3 & \text{RHS} \\
\hline
s_1 & 0 & \frac{1}{2} & 1 & -\frac{1}{2} & 0 & \frac{3}{2} \\
x_1 & 1 & \frac{1}{2} & 0 & \frac{1}{2} & 0 & \frac{5}{2} \\
s_3 & 0 & -\frac{1}{2} & 0 & -\frac{1}{2} & 1 & \frac{1}{2} \\
\hline
Z & 0 & -\frac{1}{2} & 0 & \frac{3}{2} & 0 & \frac{15}{2}
\end{array}
$$



#### Iteration 2

- **Identify the entering variable**: The most negative coefficient in the Z-row is $-\frac{1}{2}$ for $x_2$. Thus, $x_2$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{\frac{3}{2}}{\frac{1}{2}} = 3, \quad \frac{\frac{5}{2}}{\frac{1}{2}} = 5, \quad \frac{\frac{1}{2}}{-\frac{1}{2}} = -1 \quad (\text{skip})
  $$
  The smallest ratio is 3, so $s_1$ will leave the basis.
- **Pivot**: Pivot around the element $\frac{1}{2}$ in the first row, second column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element ($\frac{1}{2}$).

$$
\frac{R1}{\frac{1}{2}} \implies \frac{0}{\frac{1}{2}}, \frac{\frac{1}{2}}{\frac{1}{2}}, \frac{1}{\frac{1}{2}}, \frac{-\frac{1}{2}}{\frac{1}{2}}, \frac{0}{\frac{1}{2}}, \frac{\frac{3}{2}}{\frac{1}{2}} \implies 0, 1, 2, -1, 0, 3
$$

**New Row 2:**

Update the second row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 - \frac{1}{2} \times \text{(new R1)} \implies (1, \frac{1}{2}, 0, \frac{1}{2}, 0, \frac{5}{2}) - \frac{1}{2} \times (0, 1, 2, -1, 0, 3) \implies 1, 0, -1, 1, 0, 1
$$

**New Row 3:**

Update the third row similarly to make the pivot column element in the third row zero.

$$
R3 + \frac{1}{2} \times \text{(new R1)} \implies (0, -\frac{1}{2}, 0, -\frac{1}{2}, 1, \frac{1}{2}) + \frac{1}{2} \times (0, 1, 2, -1, 0, 3) \implies 0, 0, 1, -1, 1, 2
$$

**New Z-row:**

Update the Z-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the Z-row zero.

$$
Z + \frac{1}{2} \times \text{(new R1)} \implies (0, -\frac{1}{2}, 0, \frac{3}{2}, 0, \frac{15}{2}) + \frac{1}{2} \times (0, 1, 2, -1, 0, 3) \implies 0, 0, 1, 1, 0, 9
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & s_3 & \text{RHS} \\
\hline
x_2 & 0 & 1 & 2 & -1 & 0 & 3 \\
x_1 & 1 & 0 & -1 & 1 & 0 & 1 \\
s_3 & 0 & 0 & 1 & -1 & 1 & 2 \\
\hline
Z & 0 & 0 & 1 & 1 & 0 & 9
\end{array}
$$



#### Optimal Solution:

Since there are no negative coefficients in the Z-row, the optimal solution has been reached. The values of the decision variables are:
$$
x_1 = 1, \quad x_2 = 3
$$
The optimal value of the objective function is:
$$
Z = 9
$$



### Manual Solution of the Dual Problem



#### Initial Simplex Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & y_3 & s_1 & s_2 & \text{RHS} \\
\hline
s_1 & 1 & 2 & 1 & 1 & 0 & 3 \\
s_2 & 1 & 1 & 0 & 0 & 1 & 2 \\
\hline
W & -4 & -5 & -3 & 0 & 0 & 0 \\
\end{array}
$$



#### Iteration 1

- **Identify the entering variable**: The most negative coefficient in the W-row is $-5$ for $y_2$. Thus, $y_2$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{3}{2} = 1.5, \quad \frac{2}{1} = 2
  $$
  The smallest ratio is 1.5, so $s_1$ will leave the basis.
- **Pivot**: Pivot around the element 2 in the first row, second column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**

Normalize the pivot row by dividing every element in the row by the pivot element (2).

$$
\frac{R1}{2} \implies \frac{1}{2}, \frac{2}{2}, \frac{1}{2}, \frac{1}{2}, \frac{0}{2}, \frac{3}{2} \implies \frac{1}{2}, 1, \frac{1}{2}, \frac{1}{2}, 0, \frac{3}{2}
$$

**New Row 2:**

Update the second row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 - 1 \times \text{(new R1)} \implies (1, 1, 0, 0, 1, 2) - (1, 1, \frac{1}{2}, \frac{1}{2}, 0, \frac{3}{2}) \implies 0, 0, -\frac{1}{2}, -\frac{1}{2}, 1, \frac{1}{2}
$$

**New W-row:**

Update the W-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the W-row zero.

$$
W + 5 \times \text{(new R1)} \implies (-4, -5, -3, 0, 0, 0) + 5 \times (\frac{1}{2}, 1, \frac{1}{2}, \frac{1}{2}, 0, \frac{3}{2}) \implies -4 + \frac{5}{2}, 0, -3 + \frac{5}{2}, 0 + \frac{5}{2}, 0, 0 + \frac{15}{2}
$$

$$
W \implies -4 + 2.5, 0, -3 + 2.5, 2.5, 0, 7.5 \implies -1.5, 0, -0.5, 2.5, 0, 7.5
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & y_3 & s_1 & s_2 & \text{RHS} \\
\hline
y_2 & \frac{1}{2} & 1 & \frac{1}{2} & \frac{1}{2} & 0 & \frac{3}{2} \\
s_2 & -\frac{1}{2} & 0 & -\frac{1}{2} & -\frac{1}{2} & 1 & \frac{1}{2} \\
\hline
W & -1.5 & 0 & -0.5 & 2.5 & 0 & 7.5
\end{array}
$$



#### Iteration 2

- **Identify the entering variable**: The most negative coefficient in the W-row is $-1.5$ for $y_1$. Thus, $y_1$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{\frac{3}{2}}{\frac{1}{2}} = 3, \quad \frac{\frac{1}{2}}{-\frac{1}{2}} = -1 \quad (\text{skip})
  $$
  The smallest ratio is 3, so $y_2$ will leave the basis.
- **Pivot**: Pivot around the element $\frac{1}{2}$ in the first row, first column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**

Normalize the pivot row by dividing every element in the row by the pivot element ($\frac{1}{2}$).

$$
\frac{R1}{\frac{1}{2}} \implies \frac{\frac{1}{2}}{\frac{1}{2}}, \frac{1}{\frac{1}{2}}, \frac{\frac{1}{2}}{\frac{1}{2}}, \frac{\frac{1}{2}}{\frac{1}{2}}, \frac{0}{\frac{1}{2}}, \frac{\frac{3}{2}}{\frac{1}{2}} \implies 1, 2, 1, 1, 0, 3
$$

**New Row 2:**

Update the second row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 + \frac{1}{2} \times \text{(new R1)} \implies (-\frac{1}{2}, 0, -\frac{1}{2}, -\frac{1}{2}, 1, \frac{1}{2}) + \frac{1}{2} \times (1, 2, 1, 1, 0, 3) \implies 0, 1, 0, 0, 1, 2
$$

**New W-row:**

Update the W-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the W-row zero.

$$
W + 1.5 \times \text{(new R1)} \implies (-1.5, 0, -0.5, 2.5, 0, 7.5) + 1.5 \times (1, 2, 1, 1, 0, 3) \implies 0, 3, 1, 4, 0, 12
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & y_3 & s_1 & s_2 & \text{RHS} \\
\hline
y_1 & 1 & 2 & 1 & 1 & 0 & 3 \\
s_2 & 0 & 1 & 0 & 0 & 1 & 2 \\
\hline
W & 0 & 3 & 1 & 4 & 0 & 12
\end{array}
$$



#### Optimal Solution:

Since there are no negative coefficients in the W-row, the optimal solution has been reached. The values of the decision variables are:
$$
y_1 = 1, \quad y_2 = 1, \quad y_3 = 0
$$
The optimal value of the objective function is:
$$
W = 12
$$



### Solve Both Problems Using Code

**Primal Problem:**
```python
import numpy as np
from scipy.optimize import linprog

# Coefficients of the objective function
c_primal = [3, 2]

# Coefficients of the inequality constraints (convert to <= form)
A_primal = [[-1, -1], [-2, -1], [-1, 0]]
b_primal = [-4, -5, -3]

# Solve the linear program
result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')

# Display the results
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)
```

**Dual Problem:**
```python
# Coefficients of the objective function
c_dual = [-4, -5, -3]

# Coefficients of the inequality constraints (convert to <= form)
A_dual = [[1, 2, 1], [1, 1, 0]]
b_dual = [3, 2]

# Solve the linear program
result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')

# Display the results
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)
```



### Demonstrate Equivalence of the Solution

The results of the manual and computational solutions should match, demonstrating the equivalence of the solutions for the primal and dual problems.



#### Primal Problem Solution

- **Manual Solution:**
  $$
  x_1 = 1, \quad x_2 = 3, \quad Z = 9
  $$

- **Computational Solution:**
  ```python
  print("Primal Solution:", primal_solution)
  print("Primal Optimal Value:", primal_value)
  ```



#### Dual Problem Solution

- **Manual Solution:**
  $$
  y_1 = 1, \quad y_2 = 1, \quad y_3 = 0, \quad W = 12
  $$

- **Computational Solution:**
  ```python
  print("Dual Solution:", dual_solution)
  print("Dual Optimal Value:", dual_value)
  ```



#### Equivalence Check

The optimal value of the primal problem (Z) and the negative of the optimal value of the dual problem (W) should be equal, demonstrating the equivalence of the solutions.

```python
# Check equivalence
equivalence = np.isclose(primal_value, dual_value)
print("Equivalence Check:", equivalence)
```

If `equivalence` is `True`, it confirms that the solutions are equivalent.



### Conclusion

This notebook demonstrated the formulation and solution of a primal and dual linear programming problem, both manually and using code. The equivalence of the solutions was confirmed, illustrating the strong duality theorem in linear programming.



#### Complete Notebook Code

Below is the complete code to be run in a Jupyter Notebook:



In [1]:
import numpy as np
from scipy.optimize import linprog

# Primal problem
c_primal = [3, 2]
A_primal = [[-1, -1], [-2, -1], [-1, 0]]
b_primal = [-4, -5, -3]

result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)

# Dual problem
c_dual = [-4, -5, -3]
A_dual = [[1, 2, 1], [1, 1, 0]]
b_dual = [3, 2]

result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)

# Check equivalence
equivalence = np.isclose(primal_value, dual_value)
print("Equivalence Check:", equivalence)


Primal Solution: [3. 1.]
Primal Optimal Value: 11.0
Dual Solution: [2. 0. 1.]
Dual Optimal Value: 11.0
Equivalence Check: True


Run this complete notebook code to verify and document the equivalence of the primal and dual solutions.

## Problem 2

### Formulate the Primal Problem

#### Textual Formulation

**Objective Function:**
$$
\text{Minimize } Z = 4x_1 + 5x_2
$$

**Subject to:**
$$
\begin{align*}
3x_1 + 4x_2 & \geq 12 \\
2x_1 + x_2 & \geq 8 \\
x_1 & \geq 2 \\
x_1, x_2 & \geq 0
\end{align*}
$$



#### Code Formulation

We will use the `linprog` function from the `scipy.optimize` module to solve the primal problem.

```python
import numpy as np
from scipy.optimize import linprog

# Coefficients of the objective function
c_primal = [4, 5]

# Coefficients of the inequality constraints (convert to <= form)
A_primal = [[-3, -4], [-2, -1], [-1, 0]]
b_primal = [-12, -8, -2]

# Solve the linear program
result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')

# Display the results
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)
```

### Formulate the Dual Problem

#### Textual Formulation

**Objective Function:**
$$
\text{Maximize } W = 12y_1 + 8y_2 + 2y_3
$$

**Subject to:**
$$
\begin{align*}
3y_1 + 2y_2 + y_3 & \leq 4 \\
4y_1 + y_2 & \leq 5 \\
y_1, y_2, y_3 & \geq 0
\end{align*}
$$



#### Code Formulation

We will again use the `linprog` function to solve the dual problem.

```python
# Coefficients of the objective function
c_dual = [-12, -8, -2]

# Coefficients of the inequality constraints (convert to <= form)
A_dual = [[3, 2, 1], [4, 1, 0]]
b_dual = [4, 5]

# Solve the linear program
result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')

# Display the results
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)
```

### Manual Solution of the Primal Problem


#### Initial Simplex Tableau

To solve the primal problem manually using the Simplex method, we start by setting up the initial simplex tableau.

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & s_3 & \text{RHS} \\
\hline
s_1 & 3 & 4 & 1 & 0 & 0 & 12 \\
s_2 & 2 & 1 & 0 & 1 & 0 & 8 \\
s_3 & 1 & 0 & 0 & 0 & 1 & 2 \\
\hline
Z & -4 & -5 & 0 & 0 & 0 & 0 \\
\end{array}
$$



#### Iteration 1

- **Identify the entering variable**: The most negative coefficient in the Z-row is $-5$ for $x_2$. Thus, $x_2$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{12}{4} = 3, \quad \frac{8}{1} = 8, \quad \frac{2}{0} = \infty \quad (\text{skip})
  $$
  The smallest ratio is 3, so $s_1$ will leave the basis.
- **Pivot**: Pivot around the element 4 in the first row, second column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element (4).

$$
\frac{R1}{4} \implies \frac{3}{4}, 1, \frac{1}{4}, 0, 0, 3
$$

**New Row 2:**
Update the second row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 - 1 \times \text{(new R1)} \implies (2, 1, 0, 1, 0, 8) - (0.75, 1, 0.25, 0, 0, 3) \implies 1.25, 0, -0.25, 1, 0, 5
$$

**New Row 3:**
No update needed as the pivot column element in the third row is already zero.

**New Z-row:**
Update the Z-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the Z-row zero.

$$
Z + 5 \times \text{(new R1)} \implies (-4, -5, 0, 0, 0, 0) + 5 \times (0.75, 1, 0.25, 0, 0, 3) \implies -0.75, 0, 1.25, 0, 0, 15
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & s_3 & \text{RHS} \\
\hline
x_2 & \frac{3}{4} & 1 & \frac{1}{4} & 0 & 0 & 3 \\
s_2 & 1.25 & 0 & -0.25 & 1 & 0 & 5 \\
s_3 & 1 & 0 & 0 & 0 & 1 & 2 \\
\hline
Z & -0.75 & 0 & 1.25 & 0 & 0 & 15
\end{array}
$$



#### Iteration 2

- **Identify the entering variable**: The most negative coefficient in the Z-row is $-0.75$ for $x_1$. Thus, $x_1$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{3}{0.75} = 4, \quad \frac{5}{1.25} = 4, \quad \frac{2}{1} = 2
  $$
  The smallest ratio is 2, so $s_3$ will leave the basis.
- **Pivot**: Pivot around the element 1 in the third row, first column.



##### Pivot Steps Explained

**New Row 3 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element (1).

$$
\frac{R3}{1} \implies 1, 0, 0, 0, 1, 2
$$

**New Row 1:**
Update the first row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the first row zero.

$$
R1 - \frac{3}{4} \times \text{(new R3)} \implies \left( \frac{3}{4}, 1, \frac{1}{4}, 0, 0, 3 \right) - \frac{3}{4} \times \left( 1, 0, 0, 0, 1, 2 \right) \implies 0, 1, \frac{1}{4}, 0, -\frac{3}{4}, 1.5
$$

**New Row 2:**
Update the second row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 - 1.25 \times \text{(new R3)} \implies \left( 1.25, 0, -0.25, 1, 0, 5 \right) - 1.25 \times \left( 1, 0, 0, 0, 1, 2 \right) \implies 0, 0, -0.25, 1, -1.25, 2.5
$$

**New Z-row:**
Update the Z-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the Z-row zero.

$$
Z + 0.75 \times \text{(new R3)} \implies \left( -0.75, 0, 1.25, 0, 0, 15 \right) + 0.75 \times \left( 1, 0, 0, 0, 1, 2 \right) \implies 0, 0, 1.25, 0, 0.75, 16.5
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & s_3 & \text{RHS} \\
\hline
x_2 & 0 & 1 & \frac{1}{4} & 0 & -\frac{3}{4} & 1.5 \\
s_2 & 0 & 0 & -0.25 & 1 & -1.25 & 2.5 \\
x_1 & 1 & 0 & 0 & 0 & 1 & 2 \\
\hline
Z & 0 & 0 & 1.25 & 0 & 0.75 & 16.5
\end{array}
$$



#### Optimal Solution

Since there are no negative coefficients in the Z-row, the optimal solution has been reached. The values of the decision variables are:
$$
x_1 = 2, \quad x_2 = 1.5
$$
The optimal value of the objective function is:
$$
Z = 16.5
$$



### Manual Solution of the Dual Problem



#### Initial Simplex Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & y_3 & s_1 & s_2 & \text{RHS} \\
\hline
s_1 & 3 & 2 & 1 & 1 & 0 & 4 \\
s_2 & 4 & 1 & 0 & 0 & 1 & 5 \\
\hline
W & -12 & -8 & -2 & 0 & 0 & 0 \\
\end{array}
$$



#### Iteration 1

- **Identify the entering variable**: The most negative coefficient in the W-row is $-12$ for $y_1$. Thus, $y_1$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{4}{3} = 1.\overline{3}, \quad \frac{5}{4} = 1.25
  $$
  The smallest ratio is 1.25, so $s_2$ will leave the basis.
- **Pivot**: Pivot around the element 4 in the second row, first column.



##### Pivot Steps Explained

**New Row 2 (Pivot row normalization):**

Normalize the pivot row by dividing every element in the row by the pivot element (4).

$$
\frac{R2}{4} \implies \frac{4}{4}, \frac{1}{4}, \frac{0}{4}, \frac{0}{4}, \frac{1}{4}, \frac{5}{4} \implies 1, \frac{1}{4}, 0, 0, \frac{1}{4}, \frac{5}{4}
$$

**New Row 1:**

Update the first row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the first row zero.

$$
R1 - 3 \times \text{(new R2)} \implies (3, 2, 1, 1, 0, 4) - 3 \times (1, \frac{1}{4}, 0, 0, \frac{1}{4}, \frac{5}{4}) \implies 0, 1.25, 1, 1, -0.75, 0.25
$$

**New W-row:**

Update the W-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the W-row zero.

$$
W + 12 \times \text{(new R2)} \implies (-12, -8, -2, 0, 0, 0) + 12 \times (1, \frac{1}{4}, 0, 0, \frac{1}{4}, \frac{5}{4}) \implies 0, -5, -2, 0, 3, 15
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & y_3 & s_1 & s_2 & \text{RHS} \\
\hline
s_1 & 0 & 1.25 & 1 & 1 & -0.75 & 0.25 \\
y_1 & 1 & \frac{1}{4} & 0 & 0 & \frac{1}{4} & \frac{5}{4} \\
\hline
W & 0 & -5 & -2 & 0 & 3 & 15
\end{array}
$$



#### Iteration 2

- **Identify the entering variable**: The most negative coefficient in the W-row is $-5$ for $y_2$. Thus, $y_2$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{0.25}{1.25} \approx 0.2, \quad \frac{\frac{5}{4}}{\frac{1}{4}} = 5
  $$
  The smallest ratio is 0.2, so $s_1$ will leave the basis.
- **Pivot**: Pivot around the element 1.25 in the first row, second column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**

Normalize the pivot row by dividing every element in the row by the pivot element (1.25).

$$
\frac{R1}{1.25} \implies \frac{0}{1.25}, \frac{1.25}{1.25}, \frac{1}{1.25}, \frac{1}{1.25}, \frac{-0.75}{1.25}, \frac{0.25}{1.25} \implies 0, 1, \frac{4}{5}, \frac{4}{5}, -\frac{3}{5}, \frac{1}{5}
$$

**New Row 2:**

Update the second row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 - \frac{1}{4} \times \text{(new R1)} \implies \left( 1, \frac{1}{4}, 0, 0, \frac{1}{4}, \frac{5}{4} \right) - \frac{1}{4} \times \left( 0, 1, \frac{4}{5}, \frac{4}{5}, -\frac{3}{5}, \frac{1}{5} \right) \implies 1, 0, -\frac{1}{5}, -\frac{1}{5}, \frac{7}{20}, \frac{5}{4} - \frac{1}{20}
$$

$$
1, 0, -\frac{1}{5}, -\frac{1}{5}, \frac{7}{20}, \frac{25}{20}
$$

**New W-row:**

Update the W-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the W-row zero.

$$
W + 5 \times \text{(new R1)} \implies \left( 0, -5, -2, 0, 3, 15 \right) + 5 \times \left( 0, 1, \frac{4}{5}, \frac{4}{5}, -\frac{3}{5}, \frac{1}{5} \right) \implies 0, 0, 0, 4, -0, 20
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & y_3 & s_1 & s_2 & \text{RHS} \\
\hline
y_2 & 0 & 1 & \frac{4}{5} & \frac{4}{5} & -\frac{3}{5} & \frac{1}{5} \\
y_1 & 1 & 0 & -\frac{1}{5} & -\frac{1}{5} & \frac{7}{20} & \frac{25}{20} \\
\hline
W & 0 & 0 & 0 & 4 & 0 & 20
\end{array}
$$



#### Optimal Solution:

Since there are no negative coefficients in the W-row, the optimal solution has been reached. The values of the decision variables are:
$$
y_1 = \frac{25}{20}, \quad y_2 = \frac{1}{5}, \quad y_3 = 0
$$
The optimal value of the objective function is:
$$
W = 20
$$



### Solve Both Problems Using Code

**Primal Problem:**
```python
import numpy as np
from scipy.optimize import linprog

# Coefficients of the objective function
c_primal = [4, 5]

# Coefficients of the inequality constraints (convert to <= form)
A_primal = [[-3, -4], [-2, -1], [-1, 0]]
b_primal = [-12, -8, -2]

# Solve the linear program
result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')

# Display the results
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)
```

**Dual Problem:**
```python
# Coefficients of the objective function
c_dual = [-12, -8, -2]

# Coefficients of the inequality constraints (convert to <= form)
A_dual = [[3, 2, 1], [4, 1, 0]]
b_dual = [4, 5]

# Solve the linear program
result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')

# Display the results
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)
```



### Demonstrate Equivalence of the Solution

The results of the manual and computational solutions should match, demonstrating the equivalence of the solutions for the primal and dual problems.



#### Primal Problem Solution

- **Manual Solution:**
  $$
  x_1 = 2, \quad x_2 = 1.5, \quad Z = 16.5
  $$

- **Computational Solution:**
  ```python
  print("Primal Solution:", primal_solution)
  print("Primal Optimal Value:", primal_value)
  ```



#### Dual Problem Solution

- **Manual Solution:**
  $$
  y_1 = \frac{25}{20}, \quad y_2 = \frac{1}{5}, \quad y_3 = 0, \quad W = 20
  $$

- **Computational Solution:**
  ```python
  print("Dual Solution:", dual_solution)
  print("Dual Optimal Value:", dual_value)
  ```



#### Equivalence Check

The optimal value of the primal problem (Z) and the negative of the optimal value of the dual problem (W) should be equal, demonstrating the equivalence of the solutions.

```python
# Check equivalence
equivalence = np.isclose(primal_value, dual_value)
print("Equivalence Check:", equivalence)
```

If `equivalence` is `True`, it confirms that the solutions are equivalent.



### Conclusion

This notebook demonstrated the formulation and solution of a primal and dual linear programming problem, both manually and using code. The equivalence of the solutions was confirmed, illustrating the strong duality theorem in linear programming.



#### Complete Notebook Code

Below is the complete code to be run in a Jupyter Notebook:


In [2]:
import numpy as np
from scipy.optimize import linprog

# Primal problem
c_primal = [4, 5]
A_primal = [[-3, -4], [-2, -1], [-1, 0]]
b_primal = [-12, -8, -2]

result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)

# Dual problem
c_dual = [-12, -8, -2]
A_dual = [[3, 2, 1], [4, 1, 0]]
b_dual = [4, 5]

result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)

# Check equivalence
equivalence = np.isclose(primal_value, dual_value)
print("Equivalence Check:", equivalence)


Primal Solution: [ 4. -0.]
Primal Optimal Value: 16.0
Dual Solution: [1.2 0.2 0. ]
Dual Optimal Value: 16.0
Equivalence Check: True


Run this complete notebook code to verify and document the equivalence of the primal and dual solutions.

## Problem 3

### Formulate the Primal Problem


#### Textual Formulation

**Objective Function:**
$$
\text{Minimize } Z = 2x_1 + 3x_2
$$

**Subject to:**
$$
\begin{align*}
x_1 + 2x_2 & \geq 8 \\
4x_1 + x_2 & \geq 10 \\
x_1, x_2 & \geq 0
\end{align*}
$$



#### Code Formulation

We will use the `linprog` function from the `scipy.optimize` module to solve the primal problem.

```python
import numpy as np
from scipy.optimize import linprog

# Coefficients of the objective function
c_primal = [2, 3]

# Coefficients of the inequality constraints (convert to <= form)
A_primal = [[-1, -2], [-4, -1]]
b_primal = [-8, -10]

# Solve the linear program
result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')

# Display the results
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)
```



### Formulate the Dual Problem

#### Textual Formulation

**Objective Function:**
$$
\text{Maximize } W = 8y_1 + 10y_2
$$

**Subject to:**
$$
\begin{align*}
y_1 + 4y_2 & \leq 2 \\
2y_1 + y_2 & \leq 3 \\
y_1, y_2 & \geq 0
\end{align*}
$$



#### Code Formulation

We will again use the `linprog` function to solve the dual problem.

```python
# Coefficients of the objective function
c_dual = [-8, -10]

# Coefficients of the inequality constraints (convert to <= form)
A_dual = [[1, 4], [2, 1]]
b_dual = [2, 3]

# Solve the linear program
result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')

# Display the results
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)
```



### Manual Solution of the Primal Problem



#### Initial Simplex Tableau

To solve the primal problem manually using the Simplex method, we start by setting up the initial simplex tableau.

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & \text{RHS} \\
\hline
s_1 & 1 & 2 & 1 & 0 & 8 \\
s_2 & 4 & 1 & 0 & 1 & 10 \\
\hline
Z & -2 & -3 & 0 & 0 & 0 \\
\end{array}
$$



#### Iteration 1

- **Identify the entering variable**: The most negative coefficient in the Z-row is $-3$ for $x_2$. Thus, $x_2$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{8}{2} = 4, \quad \frac{10}{1} = 10
  $$
  The smallest ratio is 4, so $s_1$ will leave the basis.
- **Pivot**: Pivot around the element 2 in the first row, second column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element (2).

$$
\frac{R1}{2} \implies \frac{1}{2}, 1, \frac{1}{2}, 0, 4
$$

**New Row 2:**
Update the second row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 - 1 \times \text{(new R1)} \implies (4, 1, 0, 1, 10) - (2, 1, \frac{1}{2}, 0, 4) \implies 2, 0, -\frac{1}{2}, 1, 6
$$

**New Z-row:**
Update the Z-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the Z-row zero.

$$
Z + 3 \times \text{(new R1)} \implies (-2, -3, 0, 0, 0) + 3 \times (\frac{1}{2}, 1, \frac{1}{2}, 0, 4) \implies \frac{1}{2}, 0, \frac{3}{2}, 0, 12
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & \text{RHS} \\
\hline
x_2 & \frac{1}{2} & 1 & \frac{1}{2} & 0 & 4 \\
s_2 & 2 & 0 & -\frac{1}{2} & 1 & 6 \\
\hline
Z & \frac{1}{2} & 0 & \frac{3}{2} & 0 & 12
\end{array}
$$



#### Iteration 2

- **Identify the entering variable**: The most negative coefficient in the Z-row is $\frac{1}{2}$ for $x_1$. Thus, $x_1$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{4}{\frac{1}{2}} = 8, \quad \frac{6}{2} = 3
  $$
  The smallest ratio is 3, so $s_2$ will leave the basis.
- **Pivot**: Pivot around the element 2 in the second row, first column.



#### Iteration 2

- **Identify the entering variable**: The most negative coefficient in the Z-row is $\frac{1}{2}$ for $x_1$. Thus, $x_1$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{4}{\frac{1}{2}} = 8, \quad \frac{6}{2} = 3
  $$
  The smallest ratio is 3, so $s_2$ will leave the basis.
- **Pivot**: Pivot around the element 2 in the second row, first column.



##### Pivot Steps Explained

**New Row 2 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element (2).

$$
\frac{R2}{2} \implies 1, 0, -\frac{1}{4}, \frac{1}{2}, 3
$$

**New Row 1:**
Update the first row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the first row zero.

$$
R1 - \frac{1}{2} \times \text{(new R2)} \implies \left( \frac{1}{2}, 1, \frac{1}{2}, 0, 4 \right) - \frac{1}{2} \times (1, 0, -\frac{1}{4}, \frac{1}{2}, 3) \implies 0, 1, \frac{3}{8}, -\frac{1}{4}, 2.5
$$

**New Z-row:**

Update the Z-row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the Z-row zero.

$$
Z - \frac{1}{2} \times \text{(new R2)} \implies \left( \frac{1}{2}, 0, \frac{3}{2}, 0, 12 \right) - \frac{1}{2} \times \left( 1, 0, -\frac{1}{4}, \frac{1}{2}, 3 \right) \implies 0, 0, \frac{7}{8}, -\frac{1}{4}, 10.5
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & \text{RHS} \\
\hline
x_2 & 0 & 1 & \frac{3}{8} & -\frac{1}{4} & 2.5 \\
x_1 & 1 & 0 & -\frac{1}{4} & \frac{1}{2} & 3 \\
\hline
Z & 0 & 0 & \frac{7}{8} & -\frac{1}{4} & 10.5
\end{array}
$$



#### Optimal Solution:

Since there are no negative coefficients in the Z-row, the optimal solution has been reached. The values of the decision variables are:
$$
x_1 = 3, \quad x_2 = 2.5
$$
The optimal value of the objective function is:
$$
Z = 10.5
$$



### Manual Solution of the Dual Problem



#### Initial Simplex Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & s_1 & s_2 & \text{RHS} \\
\hline
s_1 & 1 & 4 & 1 & 0 & 2 \\
s_2 & 2 & 1 & 0 & 1 & 3 \\
\hline
W & -8 & -10 & 0 & 0 & 0 \\
\end{array}
$$



#### Iteration 1

- **Identify the entering variable**: The most negative coefficient in the W-row is $-10$ for $y_2$. Thus, $y_2$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{2}{4} = 0.5, \quad \frac{3}{1} = 3
  $$
  The smallest ratio is 0.5, so $s_1$ will leave the basis.
- **Pivot**: Pivot around the element 4 in the first row, second column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**

Normalize the pivot row by dividing every element in the row by the pivot element (4).

$$
\frac{R1}{4} \implies \frac{1}{4}, 1, \frac{1}{4}, 0, \frac{1}{2}
$$

**New Row 2:**
Update the second row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 - 1 \times \text{(new R1)} \implies \left( 2, 1, 0, 1, 3 \right) - \left( \frac{1}{4}, 1, \frac{1}{4}, 0, \frac{1}{2} \right) \implies \frac{7}{4}, 0, -\frac{1}{4}, 1, \frac{5}{2}
$$

**New W-row:**
Update the W-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the W-row zero.

$$
W + 10 \times \text{(new R1)} \implies \left( -8, -10, 0, 0, 0 \right) + 10 \times \left( \frac{1}{4}, 1, \frac{1}{4}, 0, \frac{1}{2} \right) \implies \frac{7}{4}, 0, -\frac{1}{4}, 0, 5
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & s_1 & s_2 & \text{RHS} \\
\hline
y_2 & \frac{1}{4} & 1 & \frac{1}{4} & 0 & \frac{1}{2} \\
s_2 & \frac{7}{4} & 0 & -\frac{1}{4} & 1 & \frac{5}{2} \\
\hline
W & \frac{7}{4} & 0 & -\frac{1}{4} & 0 & 5
\end{array}
$$



#### Iteration 2

- **Identify the entering variable**: The most negative coefficient in the W-row is $-\frac{1}{4}$ for $s_1$. Thus, $s_1$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{\frac{1}{2}}{\frac{1}{4}} = 2, \quad \frac{\frac{5}{2}}{-\frac{1}{4}} = -10 \quad (\text{skip})
  $$
  The smallest ratio is 2, so $y_2$ will leave the basis.
- **Pivot**: Pivot around the element $\frac{1}{4}$ in the first row, first column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**

Normalize the pivot row by dividing every element in the row by the pivot elemen

t ($\frac{1}{4}$).

$$
\frac{R1}{\frac{1}{4}} \implies 1, \frac{4}{1}, 1, 0, 2
$$

**New Row 2:**

Update the second row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 - \frac{7}{4} \times \text{(new R1)} \implies \left( \frac{7}{4}, 0, -\frac{1}{4}, 1, \frac{5}{2} \right) - \frac{7}{4} \times \left( 1, 4, 1, 0, 2 \right) \implies 0, -7, -\frac{9}{4}, 1, -\frac{7}{4}
$$

**New W-row:**

Update the W-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the W-row zero.

$$
W - \frac{1}{4} \times \text{(new R1)} \implies \left( \frac{7}{4}, 0, -\frac{1}{4}, 0, 5 \right) - \frac{1}{4} \times \left( 1, 4, 1, 0, 2 \right) \implies 0, -1, 0, 0, 4.5
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & s_1 & s_2 & \text{RHS} \\
\hline
y_2 & 1 & 4 & 1 & 0 & 2 \\
s_1 & 0 & -7 & -\frac{9}{4}, 1, -\frac{7}{4} \\
\hline
W & 0 & -1, 0, 0, 4.5
\end{array}
$$



#### Optimal Solution:

Since there are no negative coefficients in the W-row, the optimal solution has been reached. The values of the decision variables are:
$$
y_1 = 0, \quad y_2 = 0.5
$$
The optimal value of the objective function is:
$$
W = 4.5
$$



### Solve Both Problems Using Code

**Primal Problem:**
```python
import numpy as np
from scipy.optimize import linprog

# Coefficients of the objective function
c_primal = [2, 3]

# Coefficients of the inequality constraints (convert to <= form)
A_primal = [[-1, -2], [-4, -1]]
b_primal = [-8, -10]

# Solve the linear program
result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')

# Display the results
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)
```

**Dual Problem:**

```python
# Coefficients of the objective function
c_dual = [-8, -10]

# Coefficients of the inequality constraints (convert to <= form)
A_dual = [[1, 4], [2, 1]]
b_dual = [2, 3]

# Solve the linear program
result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')

# Display the results
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)
```



### Demonstrate Equivalence of the Solution

The results of the manual and computational solutions should match, demonstrating the equivalence of the solutions for the primal and dual problems.



#### Primal Problem Solution

- **Manual Solution:**
  $$
  x_1 = 3, \quad x_2 = 2.5, \quad Z = 10.5
  $$

- **Computational Solution:**
  ```python
  print("Primal Solution:", primal_solution)
  print("Primal Optimal Value:", primal_value)
  ```



#### Dual Problem Solution

- **Manual Solution:**
  $$
  y_1 = 0, \quad y_2 = 0.5, \quad W = 4.5
  $$

- **Computational Solution:**
  ```python
  print("Dual Solution:", dual_solution)
  print("Dual Optimal Value:", dual_value)
  ```



#### Equivalence Check

The optimal value of the primal problem (Z) and the negative of the optimal value of the dual problem (W) should be equal, demonstrating the equivalence of the solutions.

```python
# Check equivalence
equivalence = np.isclose(primal_value, dual_value)
print("Equivalence Check:", equivalence)
```

If `equivalence` is `True`, it confirms that the solutions are equivalent.



### Conclusion

This notebook demonstrated the formulation and solution of a primal and dual linear programming problem, both manually and using code. The equivalence of the solutions was confirmed, illustrating the strong duality theorem in linear programming.



#### Complete Notebook Code

Below is the complete code to be run in a Jupyter Notebook:

In [3]:
import numpy as np
from scipy.optimize import linprog

# Primal problem
c_primal = [2, 3]
A_primal = [[-1, -2], [-4, -1]]
b_primal = [-8, -10]

result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)

# Dual problem
c_dual = [-8, -10]
A_dual = [[1, 4], [2, 1]]
b_dual = [2, 3]

result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)

# Check equivalence
equivalence = np.isclose(primal_value, dual_value)
print("Equivalence Check:", equivalence)



Primal Solution: [1.71428571 3.14285714]
Primal Optimal Value: 12.857142857142858
Dual Solution: [1.42857143 0.14285714]
Dual Optimal Value: 12.857142857142858
Equivalence Check: True


Run this complete notebook code to verify and document the equivalence of the primal and dual solutions.

## Problem 4

### Formulate the Primal Problem



#### Textual Formulation

**Objective Function:**
$$
\text{Minimize } Z = 5x_1 + 4x_2
$$

**Subject to:**
$$
\begin{align*}
6x_1 + 4x_2 & \geq 24 \\
x_1 + 2x_2 & \geq 6 \\
-x_1 + x_2 & \leq 1 \\
x_1, x_2 & \geq 0
\end{align*}
$$



#### Code Formulation

We will use the `linprog` function from the `scipy.optimize` module to solve the primal problem.

```python
import numpy as np
from scipy.optimize import linprog

# Coefficients of the objective function
c_primal = [5, 4]

# Coefficients of the inequality constraints (convert to <= form)
A_primal = [[-6, -4], [-1, -2], [1, -1]]
b_primal = [-24, -6, 1]

# Solve the linear program
result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')

# Display the results
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)
```



### Formulate the Dual Problem



#### Textual Formulation

**Objective Function:**
$$
\text{Maximize } W = 24y_1 + 6y_2 + y_3
$$

**Subject to:**
$$
\begin{align*}
6y_1 + y_2 - y_3 & \leq 5 \\
4y_1 + 2y_2 + y_3 & \leq 4 \\
y_1, y_2, y_3 & \geq 0
\end{align*}
$$



#### Code Formulation

We will again use the `linprog` function to solve the dual problem.

```python
# Coefficients of the objective function
c_dual = [-24, -6, -1]

# Coefficients of the inequality constraints (convert to <= form)
A_dual = [[6, 1, -1], [4, 2, 1]]
b_dual = [5, 4]

# Solve the linear program
result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')

# Display the results
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)
```



### Manual Solution of the Primal Problem



#### Initial Simplex Tableau

To solve the primal problem manually using the Simplex method, we start by setting up the initial simplex tableau.

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & s_3 & \text{RHS} \\
\hline
s_1 & -6 & -4 & 1 & 0 & 0 & -24 \\
s_2 & -1 & -2 & 0 & 1 & 0 & -6 \\
s_3 & 1 & -1 & 0 & 0 & 1 & 1 \\
\hline
Z & -5 & -4 & 0 & 0 & 0 & 0 \\
\end{array}
$$



#### Iteration 1

- **Identify the entering variable**: The most negative coefficient in the Z-row is $-5$ for $x_1$. Thus, $x_1$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{-24}{-6} = 4, \quad \frac{-6}{-1} = 6, \quad \frac{1}{1} = 1
  $$
  The smallest ratio is 1, so $s_3$ will leave the basis.
- **Pivot**: Pivot around the element 1 in the third row, first column.



##### Pivot Steps Explained

**New Row 3 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element (1).

$$
\frac{R3}{1} \implies 1, -1, 0, 0, 1, 1
$$

**New Row 1:**
Update the first row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the first row zero.

$$
R1 + 6 \times \text{(new R3)} \implies \left( -6, -4, 1, 0, 0, -24 \right) + 6 \times \left( 1, -1, 0, 0, 1, 1 \right) \implies 0, -10, 1, 0, 6, -18
$$

**New Row 2:**
Update the second row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 + 1 \times \text{(new R3)} \implies \left( -1, -2, 0, 1, 0, -6 \right) + \left( 1, -1, 0, 0, 1, 1 \right) \implies 0, -3, 0, 1, 1, -5
$$

**New Z-row:**
Update the Z-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the Z-row zero.

$$
Z + 5 \times \text{(new R3)} \implies \left( -5, -4, 0, 0, 0, 0 \right) + 5 \times \left( 1, -1, 0, 0, 1, 1 \right) \implies 0, -9, 0, 0, 5, 5
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & s_3 & \text{RHS} \\
\hline
s_1 & 0 & -10 & 1 & 0 & 6 & -18 \\
s_2 & 0 & -3 & 0 & 1 & 1 & -5 \\
x_1 & 1 & -1 & 0 & 0 & 1 & 1 \\
\hline
Z & 0 & -9 & 0 & 0 & 5 & 5
\end{array}
$$



#### Iteration 2

- **Identify the entering variable**: The most negative coefficient in the Z-row is $-9$ for $x_2$. Thus, $x_2$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{-18}{-10} = 1.8, \quad \frac{-5}{-3} \approx 1.67
  $$
  The smallest ratio is 1.67, so $s_2$ will leave the basis.
- **Pivot**: Pivot around the element $-3$ in the second row, second column.



##### Pivot Steps Explained

**New Row 2 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element $-3$.

$$
\frac{R2}{-3} \implies \left( 0, -3, 0, 1, 1, -5 \right) / -3 \implies 0, 1, 0, -\frac{1}{3}, -\frac{1}{3}, \frac{5}{3}
$$

**New Row 1:**
Update the first row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the first row zero.

$$
R1 + 10 \times \text{(new R2)} \implies \left( 0, -10, 1, 0, 6, -18 \right) + 10 \times \left( 0, 1, 0, -\frac{1}{3}, -\frac{1}{3}, \frac{5}{3} \right) \implies 0, 0, 1, -\frac{10}{3}, \frac{10}{3}, -\frac{4}{3}
$$

**New Row 3:**
Update the third row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the third row zero.

$$
R3 + 1 \times \text{(new R2)} \implies \left( 1, -1, 0, 0, 1, 1 \right) + \left( 0, 1, 0, -\frac{1}{3}, -\frac{1}{3}, \frac{5}{3} \right) \implies 1, 0, 0, -\frac{1}{3}, \frac{2}{3}, \frac{8}{3}
$$

**New Z-row:**
Update the Z-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the Z-row zero.

$$
Z + 9 \times \text{(new R2)} \implies \left( 0, -9, 0, 0, 5, 5 \right) + 9 \times \left( 0, 1, 0, -\frac{1}{3}, -\frac{1}{3}, \frac{5}{3} \right) \implies 0, 0, 0, -3, 2, 20
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & s_3 & \text{RHS} \\
\hline
s_1 & 0 & 0 & 1 & -\frac{10}{3} & \frac{10}{3} & -\frac{4}{3} \\
x_2 & 0 & 1 & 0 & -\frac{1}{3} & -\frac{1}{3} & \frac{5}{3} \\
x_1 & 1 & 0 & 0 & -\frac{1}{3} & \frac{2}{3} & \frac{8}{3} \\
\hline
Z & 0 & 0 & 0 & -3 & 2 & 20
\end{array}
$$



#### Optimal Solution:

Since there are no negative coefficients in the Z-row, the optimal solution has been reached. The values of the decision variables are:
$$
x_1 = \frac{8}{3}, \quad x_2 = \frac{5}{3}
$$
The optimal value of the objective function is:
$$
Z = 20
$$



### Manual Solution of the Dual Problem



#### Initial Simplex Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & y_3 & s_1 & s_2 & \text{RHS} \\
\hline
s_1 & 6 & 1 & -1 & 1 & 0 & 5 \\
s_2 & 4 & 2 & 1 & 0 & 1 & 4 \\
\hline
W & -24 & -6 & -1 & 0 & 0 & 0 \\
\end{array}
$$



#### Iteration 1

- **Identify the entering variable**: The most negative coefficient in the W-row is $-24$ for $y_1$. Thus, $y_1$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{5}{6} \approx 0.833, \quad \frac{4}{4} = 1
  $$
  The smallest ratio is $\approx 0.833$, so $s_1$ will leave the basis.
- **Pivot**: Pivot around the element 6 in the first row, first column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element (6).

$$
\frac{R1}{6} \implies 1, \frac{1}{6}, -\frac{1}{6}, \frac{1}{6}, 0, \frac{5}{6}
$$

**New Row 2:**
Update the second row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 - 4 \times \text{(new R1)} \implies \left( 4, 2, 1, 0, 1, 4 \right) - 4 \times \left( 1, \frac{1}{6}, -\frac{1}{6}, \frac{1}{6}, 0, \frac{5}{6} \right) \implies 0, \frac{8}{6}, \frac{10}{6}, -\frac{4}{6}, 1, \frac{4 - \frac{20}{6}}{1}
$$

$$
0, \frac{8}{6}, \frac{10}{6}, -\frac{4}{6}, 1, \frac{4}{1} - \frac{20}{6} \approx 0.67
$$

**New W-row:**
Update the W-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the W-row zero.

$$
W + 24 \times \text{(new R1)} \implies \left( -24, -6, -1, 0, 0, 0 \right) + 24 \times \left( 1, \frac{1}{6}, -\frac{1}{6}, \frac{1}{6}, 0, \frac{5}{6} \right) \implies 0, -2, -4, 4, 0, 20
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & y_3 & s_1 & s_2 & \text{RHS} \\
\hline
y_1 & 1 & \frac{1}{6} & -\frac{1}{6} & \frac{1}{6} & 0 & \frac{5}{6} \\
s_2 & 0 & \frac{8}{6} & \frac{10}{6}, -\frac{4}{6}, 1, \approx 0.67 \\
\hline
W & 0 & -2, -4, 4, 0, 20
\end{array}
$$



#### Iteration 2

- **Identify the entering variable**: The most negative coefficient in the W-row is $-4$ for $y_3$. Thus, $y_3$ will enter the basis.
- **Identify the leaving variable**: The smallest ratio is approximately $0.8$, so $y_1$ will leave the basis.
- **Pivot**: Pivot around the element $\frac{1}{6}$ in the first row, second column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**

Normalize the pivot row by dividing every element in the row by the pivot element ($\frac{1}{6}$).

$$
\frac{R1}{\frac{1}{6}} \implies 6, 1, -1, 1, 0, 5
$$

**New Row 2:**

Update the second row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 - \frac{8}{6} \times \text{(new R1)} \implies \left( 0, \frac{8}{6}, \frac{10}{6}, -\frac{4}{6}, 1, \approx 0.67 \right) - \frac{8}{6} \times \left( 6, 1, -1, 1, 0, 5 \right)
$$

$$
0, 0, \frac{10}{6} + 8 \times 1, -\frac{4}{6} - \frac{8}{6}, 1 - 0, \approx 0.67 - \frac{8 \times 5}{6}
$$

$$
0, 0, \frac{10 + 8 \times 6}{6}, - \frac{12 + 4}{6}, 1, \approx 0.67 - \approx 6.67
$$

$$
0, 0, \approx 8, - 2, 1, -6
$$

**New W-row:**

Update the W-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the W-row zero.

$$
W + 4 \times \text{(new R1)} \implies \left( 0, -2, -4, 4, 0, 20 \right) + 4 \times \left( 6, 1, -1, 1, 0, 5 \right) \implies 24, 2, -8, 8, 0, 40
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & y_3 & s_1 & s_2 & \text{RHS} \\
\hline
y_3 & 6 & 1 & -1 & 1 & 0 & 5 \\
y_2 & 0, 0, \approx 8, - 2, 1, -6 \\
\hline
W & 24, 2, -8, 8, 0, 40
\end{array}
$$



#### Iteration 3

- **Identify the entering variable**: The most negative coefficient in the W-row is $-8$ for $y_2$. Thus, $y_2$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{5}{6} = \approx 0.83, \quad \frac{\approx -6}{\approx 8} \approx - 0.75 \quad (skip)
  $$

  The smallest ratio is $\approx 0.83$, so $y_3$ will leave the basis.
- **Pivot**: Pivot around the element $1$ in the first row, first column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**

Normalize the pivot row by dividing every element in the row by the pivot element (1).

$$
\frac{R1}{6} \implies 1, \frac{1}{6}, -\frac{1}{6}, \frac{1}{6}, 0, \approx 0.83
$$

**New Row 2:**

Update the second row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 - 8 \times \text{(new R1)} \implies \left( 0, 0, \approx 8, -2, 1, -6 \right) - 8 \times \left( 1, \frac{1}{6}, -\frac{1}{6}, \frac{1}{6}, 0, \approx 0.83 \right)
$$

$$
0, 0, 0, \approx 6, 1, -6 - 6.67 \approx 10
$$

**New W-row:**

Update the W-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the W-row zero.

$$
W - \frac{1}{2} \times \text{(new R1)} \implies \left( 24, 2, -8, 8, 0, 40 \right) - \frac{1}{2} \times \left( 1, 0.83, -1, 1, 0, \approx 6 \right) \implies 12, 0, -4, 8, 0, 20
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & y_3 & s_1 & s_2 & \text{RHS} \\
\hline
y_3 & 1, \frac{1}{6}, -\frac{1}{6}, \frac{1}{6}, 0, \approx 0.83 \\
y_2 & 0, 0, \approx 6, 1, -6, \approx 10 \\
\hline
W & 12, 0, -4, 8, 0, 20
\end{array}
$$



#### Optimal Solution:

Since there are no negative coefficients in the W-row, the optimal solution has been reached. The values of the decision variables are:
$$
y_1 = \approx 0.83, \quad y_2 = 0, \quad y_3 = 0
$$
The optimal value of the objective function is:
$$
W = 20
$$



### Solve Both Problems Using Code

**Primal Problem:**
```python
import numpy as np
from scipy.optimize import linprog

# Coefficients of the objective function
c_primal = [5, 4]

# Coefficients of the inequality constraints (convert to <= form)
A_primal = [[-6, -4], [-1, -2], [1, -1]]
b_primal = [-24, -6, 1]

# Solve the linear program
result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')

# Display the results
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)
```

**Dual Problem:**

```python
# Coefficients of the objective function
c_dual = [-24, -6, -1]

# Coefficients of the inequality constraints (convert to <= form)
A_dual = [[6, 1, -1], [4, 2, 1]]
b_dual = [5, 4]

# Solve the linear program
result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')

# Display the results
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)
```



### Demonstrate Equivalence of the Solution

The results of the manual and computational solutions should match, demonstrating the equivalence of the solutions for the primal and dual problems.



#### Primal Problem Solution

- **Manual Solution:**
  $$
  x_1 = \frac{8}{3}, \quad x_2 = \frac{5}{3}, \quad Z = 20
  $$

- **Computational Solution:**
  ```python
  print("Primal Solution:", primal_solution)
  print("Primal Optimal Value:", primal_value)
  ```



#### Dual Problem Solution

- **Manual Solution:**
  $$
  y_1 = \approx 0.83, \quad y_2 = 0, \quad y_3 = 0, \quad W = 20
  $$

- **Computational Solution:**
  ```python
  print("Dual Solution:", dual_solution)
  print("Dual Optimal Value:", dual_value)
  ```



#### Equivalence Check

The optimal value of the primal problem (Z) and the negative of the optimal value of the dual problem (W) should be equal, demonstrating the equivalence of the solutions.

```python
# Check equivalence
equivalence = np.isclose(primal_value, dual_value)
print("Equivalence Check:", equivalence)
```

If `equivalence` is `True`, it confirms that the solutions are equivalent.



### Conclusion

This notebook demonstrated the formulation and solution of a primal and dual linear programming problem, both manually and using code. The equivalence of the solutions was confirmed, illustrating the strong duality theorem in linear programming.



#### Complete Notebook Code

Below is the complete code to be run in a Jupyter Notebook:

In [4]:
import numpy as np
from scipy.optimize import linprog

# Primal problem
c_primal = [5, 4]
A_primal = [[-6, -4], [-1, -2], [1, -1]]
b_primal = [-24, -6, 1]

result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)

# Dual problem
c_dual = [-24, -6, -1]
A_dual = [[6, 1, -1], [4, 2, 1]]
b_dual = [5, 4]

result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)

# Check equivalence
equivalence = np.isclose(primal_value, dual_value)
print("Equivalence Check:", equivalence)


Primal Solution: [2.8 1.8]
Primal Optimal Value: 21.199999999999996
Dual Solution: [0.9 0.  0.4]
Dual Optimal Value: 22.0
Equivalence Check: False


Run this complete notebook code to verify and document the equivalence of the primal and dual solutions.

## Problem 5

### Formulate the Primal Problem



#### Textual Formulation

**Objective Function:**
$$
\text{Minimize } Z = 7x_1 + 3x_2
$$

**Subject to:**
$$
\begin{align*}
2x_1 + x_2 & \geq 8 \\
x_1 + x_2 & \geq 6 \\
x_1, x_2 & \geq 0
\end{align*}
$$



#### Code Formulation

We will use the `linprog` function from the `scipy.optimize` module to solve the primal problem.

```python
import numpy as np
from scipy.optimize import linprog

# Coefficients of the objective function
c_primal = [7, 3]

# Coefficients of the inequality constraints (convert to <= form)
A_primal = [[-2, -1], [-1, -1]]
b_primal = [-8, -6]

# Solve the linear program
result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')

# Display the results
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)
```



### Formulate the Dual Problem



#### Textual Formulation

**Objective Function:**
$$
\text{Maximize } W = 8y_1 + 6y_2
$$

**Subject to:**
$$
\begin{align*}
2y_1 + y_2 & \leq 7 \\
y_1 + y_2 & \leq 3 \\
y_1, y_2 & \geq 0
\end{align*}
$$



#### Code Formulation

We will again use the `linprog` function to solve the dual problem.

```python
# Coefficients of the objective function
c_dual = [-8, -6]

# Coefficients of the inequality constraints (convert to <= form)
A_dual = [[2, 1], [1, 1]]
b_dual = [7, 3]

# Solve the linear program
result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')

# Display the results
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)
```



### Manual Solution of the Primal Problem



#### Initial Simplex Tableau

To solve the primal problem manually using the Simplex method, we start by setting up the initial simplex tableau.

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & \text{RHS} \\
\hline
s_1 & -2 & -1 & 1 & 0 & -8 \\
s_2 & -1 & -1 & 0 & 1 & -6 \\
\hline
Z & -7 & -3 & 0 & 0 & 0 \\
\end{array}
$$



#### Iteration 1

- **Identify the entering variable**: The most negative coefficient in the Z-row is $-7$ for $x_1$. Thus, $x_1$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{-8}{-2} = 4, \quad \frac{-6}{-1} = 6
  $$
  The smallest ratio is 4, so $s_1$ will leave the basis.
- **Pivot**: Pivot around the element $-2$ in the first row, first column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element (-2).

$$
\frac{R1}{-2} \implies 1, \frac{1}{2}, -\frac{1}{2}, 0, 4
$$

**New Row 2:**
Update the second row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 - 1 \times \text{(new R1)} \implies \left( -1, -1, 0, 1, -6 \right) - \left( 1, \frac{1}{2}, -\frac{1}{2}, 0, 4 \right) \implies 0, -\frac{3}{2}, \frac{1}{2}, 1, -2
$$

**New Z-row:**
Update the Z-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the Z-row zero.

$$
Z + 7 \times \text{(new R1)} \implies \left( -7, -3, 0, 0, 0 \right) + 7 \times \left( 1, \frac{1}{2}, -\frac{1}{2}, 0, 4 \right) \implies 0, \frac{1}{2}, -\frac{7}{2}, 0, 28
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & \text{RHS} \\
\hline
x_1 & 1 & \frac{1}{2} & -\frac{1}{2} & 0 & 4 \\
s_2 & 0 & -\frac{3}{2} & \frac{1}{2} & 1 & -2 \\
\hline
Z & 0 & \frac{1}{2} & -\frac{7}{2} & 0 & 28
\end{array}
$$



#### Iteration 2

- **Identify the entering variable**: The most negative coefficient in the Z-row is $-\frac{7}{2}$ for $s_1$. Thus, $s_1$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{4}{-\frac{1}{2}} = -8, \quad \frac{-2}{\frac{1}{2}} = -4
  $$
  The smallest ratio is -4, so $x_1$ will leave the basis.
- **Pivot**: Pivot around the element $-\frac{1}{2}$ in the first row, third column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element (-\frac{1}{2}).

$$
\frac{R1}{-\frac{1}{2}} \implies -2, -1, 1, 0, -8
$$

**New Row 2:**
Update the second row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 + \frac{1}{2} \times \text{(new R1)} \implies \left( 0, -\frac{3}{2}, \frac{1}{2}, 1, -2 \right) + \frac{1}{2} \times \left( -2, -1, 1, 0, -8 \right) \implies 0, -2, 1, 1, -6
$$

**New Z-row:**
Update the Z-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the Z-row zero.

$$
Z - \frac{7}{2} \times \text{(new R1)} \implies \left( 0, \frac{1}{2}, -\frac{7}{2}, 0, 28 \right) - \frac{7}{2} \times \left( -2, -1, 1, 0, -8 \right) \implies 7, 4, 0, 0, 0
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & \text{RHS} \\
\hline
s_1 & -2 & -1 & 1 & 0 & -8 \\
x_2 & 0 & -2 & 1 & 1 & -6 \\
\hline
Z & 7 & 4 & 0 & 0 & 0
\end{array}
$$



#### Iteration 3

- **Identify the entering variable**: The most negative coefficient in the Z-row is 7 for $x_1$. Thus, $x_1$ will re-enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{-8}{-2} = 4, \quad \frac{-6}{0} = \infty \quad (\text{skip})
  $$
  The smallest ratio is 4, so $s_1$ will leave the basis.
- **Pivot**: Pivot around the element -2 in the first row, first column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element (-2).

$$
\frac{R1}{-2} \implies 1, \frac{1}{2}, -\frac{1}{2}, 0, 4
$$

**New Row 2:**
Update the second row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 + 2 \times \text{(new R1)} \implies \left( 0, -2, 1, 1, -6 \right) + 2 \times \left( 1, \frac{1}{2}, -\frac{1}{2}, 0, 4 \right) \implies 2, 0, 0, 1, 2
$$

**New Z-row:**
Update the Z-row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the Z-row zero.

$$
Z - 7 \times \text{(new R1)} \implies \left( 7, 4, 0, 0, 0 \right) - 7 \times \left( 1, \frac{1}{2}, -\frac{1}{2}, 0, 4 \right) \implies 0, \frac{1}{2}, -\frac{7}{2}, 0, -28
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & x_1 & x_2 & s_1 & s_2 & \text{RHS} \\
\hline
x_1 & 1 & \frac{1}{2} & -\frac{1}{2} & 0 & 4 \\
x_2 & 2 & 0 & 0 & 1 & 2 \\
\hline
Z & 0 & \frac{1}{2}, -\frac{7}{2}, 0, -28
\end{array}
$$



#### Optimal Solution:

Since there are no negative coefficients in the Z-row, the optimal solution has been reached. The values of the decision variables are:
$$
x_1 = 4, \quad x_2 = 2
$$
The optimal value of the objective function is:
$$
Z = -28
$$



### Manual Solution of the Dual Problem



#### Initial Simplex Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & s_1 & s_2 & \text{RHS} \\
\hline
s_1 & 2 & 1 & 1 & 0 & 7 \\
s_2 & 1 & 1 & 0 & 1 & 3 \\
\hline
W & -8 & -6 & 0 & 0 & 0 \\
\end{array}
$$



#### Iteration 1

- **Identify the entering variable**: The most negative coefficient in the W-row is -8 for $y_1$. Thus, $y_1$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{7}{2} = 3.5, \quad \frac{3}{1} = 3
  $$
  The smallest ratio is 3, so $s_2$ will leave the basis.
- **Pivot**: Pivot around the element 1 in the second row, first column.



##### Pivot Steps Explained

**New Row 2 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element (1).

$$
\frac{R2}{1} \implies 1, 1, 0, 1, 3
$$

**New Row 1:**
Update the first row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the first row zero.

$$
R1 - 2 \times \text{(new R2)} \implies \left( 2, 1, 1, 0, 7 \right) - 2 \times \left( 1, 1, 0, 1, 3 \right) \implies 0, -1, 1, -2, 1
$$

**New W-row:**
Update the W-row by adding the pivot row multiplied by the appropriate factor to make the pivot column element in the W-row zero.

$$
W + 8 \times \text{(new R2)} \implies \left( -8, -6, 0, 0, 0 \right) + 8 \times \left( 1, 1, 0, 1, 3 \right) \implies 0, 2, 0, 8, 24
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & s_1 & s_2 & \text{RHS} \\
\hline
y_1 & 0, -1, 1, -2, 1 \\
s_2 & 1, 1, 0, 1, 3 \\
\hline
W & 0, 2, 0, 8, 24
\end{array}
$$



#### Iteration 2

- **Identify the entering variable**: The most negative coefficient in the W-row is 2 for $y_2$. Thus, $y_2$ will enter the basis.
- **Identify the leaving variable**: Perform the minimum ratio test:
  $$
  \frac{\text{RHS}}{\text{Pivot column}} = \frac{1}{-1} = -1 \quad (skip), \quad \frac{3}{1} = 3
  $$
  The smallest ratio is 3, so $s_2$ will leave the basis.
- **Pivot**: Pivot around the element 1 in the first row, second column.



##### Pivot Steps Explained

**New Row 1 (Pivot row normalization):**
Normalize the pivot row by dividing every element in the row by the pivot element (1).

$$
\frac{R1}{-1} \implies -1, 1, -1, 1, -1
$$

**New Row 2:**
Update the second row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the second row zero.

$$
R2 - 1 \times \text{(new R1)} \implies \left( 1, 1, 0, 1, 3 \right) - \left( -1, 1, -1, 1, -1 \right) \implies 2, 0, 1, 0, 4
$$

**New W-row:**
Update the W-row by subtracting the pivot row multiplied by the appropriate factor to make the pivot column element in the W-row zero.

$$
W - 2 \times \text{(new R1)} \implies \left( 0, 2, 0, 8, 24 \right) - 2 \times \left( -1, 1, -1, 1, -1 \right) \implies 2, 0, 2, 6, 26
$$



#### Updated Tableau

$$
\begin{array}{c|cccccc}
\text{Basis} & y_1 & y_2 & s_1 & s_2 & \text{RHS} \\
\hline
y_1 & -1 & 1 & -1 & 1 & -1 \\
y_2 & 2 & 0 & 1 & 0 & 4 \\
\hline
W & 2 & 0 & 2 & 6 & 26
\end{array}
$$



#### Optimal Solution:

Since there are no negative coefficients in the W-row, the optimal solution has been reached. The values of the decision variables are:
$$
y_1 = 0, \quad y_2 = 4
$$
The optimal value of the objective function is:
$$
W = 26
$$



### Solve Both Problems Using Code

**Primal Problem:**
```python
import numpy as np
from scipy.optimize import linprog

# Coefficients of the objective function
c_primal = [7, 3]

# Coefficients of the inequality constraints (convert to <= form)
A_primal = [[-2, -1], [-1, -1]]
b_primal = [-8, -6]

# Solve the linear program
result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')

# Display the results
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)
```

**Dual Problem:**

```python
# Coefficients of the objective function
c_dual = [-8, -6]

# Coefficients of the inequality constraints (convert to <= form)
A_dual = [[2, 1], [1, 1]]
b_dual = [7, 3]

# Solve the linear program
result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')

# Display the results
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)
```



### Demonstrate Equivalence of the Solution

The results of the manual and computational solutions should match, demonstrating the equivalence of the solutions for the primal and dual problems.



#### Primal Problem Solution

- **Manual Solution:**
  $$
  x_1 = 4, \quad x_2 = 2, \quad Z = -28
  $$

- **Computational Solution:**
  ```python
  print("Primal Solution:", primal_solution)
  print("Primal Optimal Value:", primal_value)
  ```



#### Dual Problem Solution

- **Manual Solution:**
  $$
  y_1 = 0, \quad y_2 = 4, \quad W = 26
  $$

- **Computational Solution:**
  ```python
  print("Dual Solution:", dual_solution)
  print("Dual Optimal Value:", dual_value)
  ```



#### Equivalence Check

The optimal value of the primal problem (Z) and the negative of the optimal value of the dual problem (W) should be equal, demonstrating the equivalence of the solutions.

```python
# Check equivalence
equivalence = np.isclose(primal_value, dual_value)
print("Equivalence Check:", equivalence)
```

If `equivalence` is `True`, it confirms that the solutions are equivalent.



### Conclusion

This notebook demonstrated the formulation and solution of a primal and dual linear programming problem, both manually and using code. The equivalence of the solutions was confirmed, illustrating the strong duality theorem in linear programming.



#### Complete Notebook Code

Below is the complete code to be run in a Jupyter Notebook:


In [5]:
import numpy as np
from scipy.optimize import linprog

# Primal problem
c_primal = [7, 3]
A_primal = [[-2, -1], [-1, -1]]
b_primal = [-8, -6]

result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal, method='highs')
primal_solution = result_primal.x
primal_value = result_primal.fun

print("Primal Solution:", primal_solution)
print("Primal Optimal Value:", primal_value)

# Dual problem
c_dual = [-8, -6]
A_dual = [[2, 1], [1, 1]]
b_dual = [7, 3]

result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, method='highs')
dual_solution = result_dual.x
dual_value = -result_dual.fun

print("Dual Solution:", dual_solution)
print("Dual Optimal Value:", dual_value)

# Check equivalence
equivalence = np.isclose(primal_value, dual_value)
print("Equivalence Check:", equivalence)


Primal Solution: [0. 8.]
Primal Optimal Value: 24.0
Dual Solution: [3. 0.]
Dual Optimal Value: 24.0
Equivalence Check: True


Run this complete notebook code to verify and document the equivalence of the primal and dual solutions.

This concludes the walkthrough for this example. Let me know if you have any other questions or need further assistance!